## Implement with MNIST

0 : fake <br>
1 : real <br>

In [13]:
import tensorflow as tf
import numpy as np
import matplotlib 

from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [2]:
num_epoch = 100000
batch_size = 64
num_input = 28*28
num_latent_variable = 100
num_hidden = 128
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, num_input])
z = tf.placeholder(tf.float32, [None, num_latent_variable])

# Generator 변수들 설정
# 100 -> 128 -> 784
with tf.variable_scope('generator'):
    # 히든 레이어 파라미터
    G_W1 = tf.Variable(tf.random_normal(shape=[num_latent_variable, num_hidden], stddev=5e-2))
    G_b1 = tf.Variable(tf.constant(0.1, shape=[num_hidden]))

    # 아웃풋 레이터 파라미터
    G_W2 = tf.Variable(tf.random_normal(shape=[num_hidden, num_input], stddev=5e-2))
    G_b2 = tf.Variable(tf.constant(0.1, shape=[num_input]))

# Discriminator 변수들 설정
# 784 -> 128 -> 1
with tf.variable_scope('discriminator'):
    # 히든 레이어 파라미터
    D_W1 = tf.Variable(tf.random_normal(shape=[num_input, num_hidden], stddev=5e-2))
    D_b1 = tf.Variable(tf.constant(0.1, shape=[num_hidden]))

    # 아웃풋 레이어 파라미터
    D_W2 = tf.Variable(tf.random_normal(shape=[num_hidden, 1], stddev=5e-2))
    D_b2 = tf.Variable(tf.constant(0.1, shape=[1]))

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# Input : z from latent Variable, output : 생성된 MNIST 이미지
def build_generator(X):
    hidden_layer = tf.nn.relu((tf.matmul(X, G_W1) + G_b1))
    output_layer = tf.matmul(hidden_layer, G_W2) + G_b2
    generated_mnist_image = tf.nn.sigmoid(output_layer)
    return generated_mnist_image


# input : 인풋 이미지, output : 예측값(0 or 1), logits : sigmoid를 씌우기 전 출력값
def build_discriminator(X):
    hidden_layer = tf.nn.relu((tf.matmul(X, D_W1) + D_b1))
    logits = tf.matmul(hidden_layer, D_W2) + D_b2
    predicted_value = tf.nn.sigmoid(logits)
    return predicted_value, logits

Discriminator의 손실함수
- 실제 이미지일 확률이 높으면 1에 가까운 값을 반환하고
- 가짜 이미지일 확률이 높으면 0에 가까운 값을 반환하도록 구현

In [4]:
# 생성자를 선언
G = build_generator(z)

# 구별자를 선언
D_real, D_real_logits = build_discriminator(X)  # D(x)
D_fake, D_fake_logits = build_discriminator(G)  # D(G(z))

In [5]:
# Discriminator의 cost function 선언
d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_real_logits, labels=tf.ones_like(D_real_logits)))  # log(D(x))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.zeros_like(D_fake_logits))) # log(1-D(G(z)))
d_loss = d_loss_real + d_loss_fake  # log(D(x)) + log(1-D(G(z)))

In [6]:
# Generator의 cost function 선언
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, labels=tf.ones_like(D_fake_logits)))  # log(D(G(z)))

## Training Process

* mini-batch학습마다 구성
* 우선 매 step마다 mini batch로 G,D 샘플링
    * Dataset으로부터 얻은 x의 mini-batch
    * Uniform distribution으로부터 얻은 latent z의 mini-batch
* 2개의 Gradient step이 동시에 일어남
    * J_D를 감소시키는 방향으로 D-param만을 업데이트하는 gradient descent
    * J_G를 감소시키는 방향으로 G_param만을 업데이트하는 gradient descent

In [7]:
# 전체 파라미터를 Discriminator와 관련된 파라미터와 Generator로 관련된 파라미터로 나눔
tvar = tf.trainable_variables()
dvar = [var for var in tvar if 'discriminator' in var.name]
gvar = [var for var in tvar if 'generator' in var.name]

# Discriminator와 Generator의 Optimizer를 정의함
d_train_step = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=dvar)
g_train_step = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=gvar)

In [8]:
# 데이터 다운로드
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [10]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
num_img = 0
if not os.path.exists('generated_output/'):
    os.makedirs('generated_output/')

In [17]:
print("9".zfill(3))

009


In [18]:
def plot(samples):
    fig = plt.figure(figsize=(8,8))
    gs = gridspec.GridSpec(8,8)
    gs.update(wspace=0.05, hspace=0.05)

    for i,sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        plt.imshow(sample.reshape(28,28))
    return fig

In [20]:
# 학습 시작
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    # num_epoch 횟수만큼 최적화를 수행함
    for i in range(num_epoch):
        batch_X, _ = mnist.train.next_batch(batch_size)
        
        # latent Variable의 인풋으로 사용한 noise를 Uniform Distribution에서 batch_size개 만큼 샘플링함
        batch_noise = np.random.uniform(-1., 1., [batch_size, 100])

        # 500번 반복할 때마다 학습된 G를 통해 생성된 이미지를 저장함
        if i%500 == 0:
            samples = sess.run(G, feed_dict={z:np.random.uniform(-1., 1., [64, 100])})
            fig = plot(samples)
            plt.savefig('generated_output/%s.png'%str(num_img).zfill(3), bbox_inches='tight')
            num_img += 1
            plt.close(fig)

        # Discriminator 최적화를 수행하고 Discriminator의 손실함수를 return
        _, d_loss_print = sess.run([d_train_step, d_loss], feed_dict={X:batch_X, z:batch_noise})

        # Generator 최적화를 수행하고 Generator 손실함수를 return
        _, g_loss_print = sess.run([g_train_step, g_loss], feed_dict={z: batch_noise})

        # 100번 반복할 때마다 Discriminator의 손실함수와 Generator의 손실함수를 출력한다
        if i%100 == 0:
            print('반복(Epoch) : %d, Generator 손실함수(g_loss): %f, Discriminator 손실함수 (d_loss): %f'%(i, g_loss_print, d_loss_print))


반복(Epoch) : 0, Generator 손실함수(g_loss): 1.205080, Discriminator 손실함수 (d_loss): 1.356527
반복(Epoch) : 100, Generator 손실함수(g_loss): 3.328565, Discriminator 손실함수 (d_loss): 0.263443
반복(Epoch) : 200, Generator 손실함수(g_loss): 4.272277, Discriminator 손실함수 (d_loss): 0.112441
반복(Epoch) : 300, Generator 손실함수(g_loss): 4.915137, Discriminator 손실함수 (d_loss): 0.027982
반복(Epoch) : 400, Generator 손실함수(g_loss): 4.731428, Discriminator 손실함수 (d_loss): 0.034692
반복(Epoch) : 500, Generator 손실함수(g_loss): 5.428992, Discriminator 손실함수 (d_loss): 0.020949
반복(Epoch) : 600, Generator 손실함수(g_loss): 9.594582, Discriminator 손실함수 (d_loss): 0.010140
반복(Epoch) : 700, Generator 손실함수(g_loss): 9.954645, Discriminator 손실함수 (d_loss): 0.009327
반복(Epoch) : 800, Generator 손실함수(g_loss): 7.771447, Discriminator 손실함수 (d_loss): 0.015167
반복(Epoch) : 900, Generator 손실함수(g_loss): 15.141100, Discriminator 손실함수 (d_loss): 0.014185
반복(Epoch) : 1000, Generator 손실함수(g_loss): 11.284182, Discriminator 손실함수 (d_loss): 0.029712
반복(Epoch) : 1100, Ge